<a href="https://colab.research.google.com/github/vinayshanbhag/covid19/blob/master/corona_virus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import folium
from datetime import datetime
from folium import plugins
from functools import reduce

In [0]:
confirmed_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"
recovered_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv"
deaths_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv"
df_confirmed = pd.read_csv(confirmed_url)
df_recovered = pd.read_csv(recovered_url)
df_deaths = pd.read_csv(deaths_url)

In [0]:
def restructure(df):
  df.columns=list(df.columns[:4])+[datetime.strptime(i,"%m/%d/%y").strftime("%Y-%m-%d") for i in df.columns[4:]]
  df = df.melt(id_vars=["Province/State", "Country/Region","Lat","Long"], 
        var_name="Date", 
        value_name="Value")
  df.columns = ["state","country","latitude","longitude","date","value"]
  df.date = pd.to_datetime(df.date)
  return df

In [0]:
df_confirmed = restructure(df_confirmed)
df_recovered = restructure(df_recovered)
df_deaths = restructure(df_deaths)

In [0]:
df_active = (df_confirmed.set_index(['state','country','latitude','longitude','date'])-\
df_recovered.set_index(['state','country','latitude','longitude','date'])-\
df_deaths.set_index(['state','country','latitude','longitude','date'])).reset_index()

In [0]:
def map_data(df):
  points=[]
  for dt in [d.__str__().split("T")[0] for d in sorted(df.date.unique())]:
    tmp = df[(df.date==dt)&(df.value>0)]
    for _, x in tmp.iterrows():
      feature={
                'type': 'Feature',
                'geometry': {
                    'coordinates': [x.longitude,x.latitude],
                    'type': 'Point'
                },
                'properties': {
                    'icon': 'circle',
                    'iconstyle': {
                        'radius': np.log10(x.value)*3 if x.value else 0,
                        'stroke':False,
                        'opacity':0.5
                    },
                    'style': {'color': 'red'},
                    'times': [f'{dt}T00:00:00', f'{dt}T23:59:59']
                }
              }
      points.append(feature)



  data = {
    "type": "FeatureCollection",
    "features": points
  }

  m = folium.Map([41.914393, 12.499041], tiles = "CartoDB positron",zoom_start=3, control_scale=True, attr='')
  m.add_child(plugins.TimestampedGeoJson(data
                                        ,period='P1D'
                                        ,transition_time=200
                                        ,duration='PT1H'
                                        ,add_last_point=False)
  )
  return m

In [7]:
map_data(df_confirmed)

In [0]:
#map_data(df_deaths)

In [0]:
#map_data(df_active)

In [0]:
df_confirmed_by_country = df_confirmed[df_confirmed.date==df_confirmed.date.max()][['country','value']].groupby(['country']).agg(sum).reset_index().rename(columns={"value":"confirmed"})
df_recovered_by_country = df_recovered[df_recovered.date==df_recovered.date.max()][['country','value']].groupby(['country']).agg(sum).reset_index().rename(columns={"value":"recovered"})
df_deaths_by_country = df_deaths[df_deaths.date==df_deaths.date.max()][['country','value']].groupby(['country']).agg(sum).reset_index().rename(columns={"value":"deaths"})
df_active_by_country = df_active[df_active.date==df_active.date.max()][['country','value']].groupby(['country']).agg(sum).reset_index().rename(columns={"value":"active"})


In [0]:
df_stats_by_country = reduce(lambda  left,right: pd.merge(left,right,on=['country'],how='outer'), [df_confirmed_by_country,df_recovered_by_country,df_deaths_by_country,df_active_by_country])

In [12]:
pd.options.display.max_rows=150
df_stats_by_country.sort_values(by="confirmed",ascending=False)

,country,confirmed,recovered,deaths,active
65,Mainland China,80757.0,60106.0,3136.0,17515.0
54,Italy,10149.0,724.0,631.0,8794.0
50,Iran (Islamic Republic of),8042.0,2731.0,291.0,5020.0
92,Republic of Korea,7513.0,247.0,54.0,7212.0
36,France,1784.0,12.0,33.0,1739.0
108,Spain,1695.0,32.0,35.0,1628.0
39,Germany,1457.0,18.0,2.0,1437.0
119,US,959.0,8.0,28.0,923.0
82,Others,696.0,40.0,6.0,650.0
55,Japan,581.0,101.0,10.0,470.0
